In [1]:
import geopandas as gp
import os
import mapboxgl
from mapboxgl.viz import *
from mapboxgl.utils import *
import colorsys
import pandas as pd
import json

In [2]:
token = 'pk.eyJ1IjoiZGVyaWxhbHdpbmRhIiwiYSI6ImNqZmo2MjNiYTBkb3oyd3E4eGc3aDJtNzUifQ.nFfjYL-emL-rnxzpLgwOGg'
d = {'id_prov': [11,51,36,17,34,31,75,15,32,33,35,61,63,62,64,65,19,21,18,81,82,52,53,91,92,14,76,73,72,74,71,13,16,12], 
     'nama_prov': ['ACEH','BALI','BANTEN','BENGKULU','DAERAH ISTIMEWA YOGYAKARTA','DKI JAKARTA','GORONTALO','JAMBI','JAWA BARAT','JAWA TENGAH','JAWA TIMUR','KALIMANTAN BARAT','KALIMANTAN SELATAN','KALIMANTAN TENGAH','KALIMANTAN TIMUR','KALIMANTAN UTARA','KEPULAUAN BANGKA BELITUNG','KEPULAUAN RIAU','LAMPUNG','MALUKU','MALUKU UTARA','NUSA TENGGARA BARAT','NUSA TENGGARA TIMUR','PAPUA BARAT','PAPUA','RIAU','SULAWESI BARAT','SULAWESI SELATAN','SULAWESI TENGAH','SULAWESI TENGGARA','SULAWESI UTARA','SUMATERA BARAT','SUMATERA SELATAN','SUMATERA UTARA',
]}
provdf = pd.DataFrame(data=d)

def create_colors(kabkot_list):
    N = len(kabkot_list)
    HSV_tuples = [(x * 1.0 / N, 0.5, 0.5) for x in range(N)]
    rgb_out = []
    for index,rgb in enumerate(HSV_tuples):
        rgb = map(lambda x: int(x * 255), colorsys.hsv_to_rgb(*rgb))
        rgb_out.append([kabkot_list[index],'rgb' + str(tuple(rgb))])
    return rgb_out

def gdf_to_geojson(gdf, properties=None, filename=None):
    """Serialize a GeoPandas dataframe to a geojson format Python dictionary / file
    """
    if not properties:
        # if no properties are selected, use all properties in dataframe
        properties = [c for c in gdf.columns if c not in ["geometry"]]

    gdf_out = gdf[['geometry'] + properties or []]

    geojson_str = gdf_out.to_json()

    if filename:
        with codecs.open(filename, "w", "utf-8-sig") as f:
            f.write(geojson_str)
        return None
    else:
        return json.loads(geojson_str)

def validate_sesuai(sesuai):
    if not sesuai :
        return "Y"
    elif sesuai == "Y" or sesuai == "y" :
        return "Y"
    elif sesuai == "N" or sesuai == "n":
        return "N"
    else :
        print("Jawab Y atau N")
        return False

def validate_index_input(inputted, intersected_df) :
    if inputted :
        index_input = choose_df.loc[choose_df.index == int(inputted)]
        if len(index_input) <= 0 :
            print("Pilih dari list yang yang tertera ")
            return False
        else :
            return True
    else :
        return True

def validate_id_prov(id_prov):
    if not id_prov :
        print("Masukkan ID provinsi")
        return False
    nama_prov = provdf.query("id_prov == "+ str(id_prov))["nama_prov"].max()
    if pd.isnull(nama_prov) :
        print("Tidak ada provinsi dengan ID tersebut")
        return False
    return nama_prov

def new_show (self, **kwargs):
    
        html = self.create_html(**kwargs)
        map_html = self.as_iframe(html)
        return map_html
#         display(HTML(map_html))

mapboxgl.viz.MapViz.show = new_show

In [3]:
indo_geo_kem = gp.read_file('indonesia_kem_full.shp')

In [ ]:
prov_validated = False
display(HTML(provdf.to_html()))

while not prov_validated :
    id_prov = int(input("ID Provinsi : "))
    prov_validated = validate_id_prov(id_prov)

if id_prov == 92 :
    id_prov_bps = 94
else :
    id_prov_bps = id_prov
    
if prov_validated :    
    nama_prov = prov_validated
    nama_prov_no_space = re.sub(r'\s','',nama_prov)
    
prov_geo_kem = indo_geo_kem.loc[indo_geo_kem['ID_PROV'] == int(id_prov)]
prov_need_to_be_fixed = gp.read_file('./'+nama_prov+'/'+nama_prov_no_space+'_final.shp')
duplicated_df = pd.read_csv('./'+nama_prov+'/'+nama_prov_no_space+'_perlu_dicek.csv',sep=',')

for index, row in duplicated_df.iterrows() :
    queried = prov_need_to_be_fixed.loc[prov_need_to_be_fixed['FID'] == int(row["FID"])]
    bps_info = queried[["FID","KABKOT","KECAMATAN","DESA"]].to_html()
    kemendagri_info = queried[["NAMA_KABUP","NAMA_KECAM","NAMA_DESA"]].to_html()
    display(HTML('<div style="width:48%;display:block;position:absolute;text-align:left;">BPS '+ bps_info +'</div>' + '<div style="width:48%;position:relative;left:50%;text-align:left;"> KEMENDAGRI'+ kemendagri_info +'</div>'))

    sim = queried.iloc[0]["SIMILARITY"]
    if sim == 1 :
        display(HTML('<h3 style="color:green;text-align:center">Kesamaan : 100% </h3>'))
    elif sim < 1 and sim >= 0.8 :
        display(HTML('<h3 style="color:orange;text-align:center">Kesamaan : '+ str(sim * 100) +'%  </h3>'))
    else :
        display(HTML('<h3 style="color:red;text-align:center">Kesamaan : '+ str(sim * 100) +'%  </h3>'))

    validated = False

    while not validated :
        sesuai = str(input("Apakah data sudah sesuai ? (Y/N) (default Y) : "))
        validated = validate_sesuai(sesuai)
    
    if validated == "Y" or validated == "y" :
        continue
    else :
        queried_geojson = gdf_to_geojson(queried)
        minx = queried.bounds["minx"].min()
        miny = queried.bounds["miny"].min()
        maxx = queried.bounds["maxx"].max()
        maxy = queried.bounds["maxy"].max()
        boundingbox = [minx,miny,maxx,maxy]
        centerx = minx + ((maxx - minx) / 2)
        centery = miny + ((maxy - miny) / 2)
        center = [centerx , centery]
        desa_list = queried["DESA"].dropna().unique()
        match_color_stops = create_colors(list(desa_list))
        viz =  ChoroplethViz(queried_geojson,
                        div_id = "peta1",
                        access_token=token,
                        color_default='rgb(255,0,0)',
                        color_property='KABKOT', 
                        color_stops=match_color_stops, 
                        color_function_type='match', 
                        opacity=0.8, 
                        center=(center[0], center[1]), 
                        zoom=13, 
                        below_layer='waterway-label')
        intersections = gp.sjoin(queried, prov_geo_kem, how="left", op='intersects')
        list_interstion_ids = intersections["ID"].dropna().unique()
        intersection_spatial = prov_geo_kem[prov_geo_kem.ID.isin(list_interstion_ids)]
        intersection_spatial_geojson = gdf_to_geojson(intersection_spatial)
        intesected_nama_desa = intersection_spatial["NAMA_DESA"].dropna().unique()
        intersected_color_stops = create_colors(list(intesected_nama_desa))
        intersected_minx = intersection_spatial.bounds["minx"].min()
        intersected_miny = intersection_spatial.bounds["miny"].min()
        intersected_maxx = intersection_spatial.bounds["maxx"].max()
        intersected_maxy = intersection_spatial.bounds["maxy"].max()
        intersected_boundingbox = [minx,miny,maxx,maxy]
        intersected_centerx = intersected_minx + ((intersected_maxx - intersected_minx) / 2)
        intersected_centery = intersected_miny + ((intersected_maxy - intersected_miny) / 2)
        intersected_center = [intersected_centerx , intersected_centery]
        intersected_viz =  ChoroplethViz(intersection_spatial_geojson,
                        div_id = "peta3",
                        access_token=token,
                        color_default='rgb(255,0,0)',
                        color_property='NAMA_DESA', 
                        color_stops=intersected_color_stops, 
                        color_function_type='match', 
                        opacity=0.8, 
                        center=(intersected_center[0], intersected_center[1]), 
                        zoom=12, 
                        below_layer='waterway-label')
        display(HTML('<div style="width:48%;display:block;position:absolute;text-align:center;">BPS '+ viz.show() +'</div>' + '<div style="width:48%;position:relative;left:50%;text-align:center;"> KEMENDAGRI'+ intersected_viz.show() +'</div>'))
        choose_df = intersection_spatial[["ID","NAMA_KABUP","NAMA_KECAM","NAMA_DESA"]].reset_index()
        choose_df.index = choose_df.index + 1
        display(HTML(choose_df.to_html()))
        validate_input = False

        while not validate_input :
            inputted = input("Pilih nilai yang akan dimasukkan sebagai attribut (biarkan kosong apabila attribut akan dikosongkan) : ")
            validate_input = validate_index_input(inputted, choose_df)
        
        intersection_columns = prov_need_to_be_fixed.columns.intersection(indo_geo_kem.drop(['geometry'], axis=1).columns)
              
        if inputted :
            print(inputted)
            validated_desa = choose_df.loc[choose_df.index == int(inputted)]
            desa_from_kemendagri = indo_geo_kem.loc[indo_geo_kem['ID'] == validated_desa.iloc[0]["ID"]]
            prov_need_to_be_fixed.loc[prov_need_to_be_fixed['FID'] == int(row["FID"]), intersection_columns] = desa_from_kemendagri.drop(['geometry'], axis=1)[intersection_columns].values
        else :
            prov_need_to_be_fixed.loc[prov_need_to_be_fixed['FID'] == int(row["FID"]), intersection_columns] = None
directory = '..//BPS Hasil Final (Sudah Dicek)//'+nama_prov+'/'
if not os.path.exists(directory):
    os.makedirs(directory)
prov_need_to_be_fixed.to_file(directory+nama_prov_no_space+'_checked.shp');


,id_prov,nama_prov
0,11,ACEH
1,51,BALI
2,36,BANTEN
3,17,BENGKULU
4,34,DAERAH ISTIMEWA YOGYAKARTA
5,31,DKI JAKARTA
6,75,GORONTALO
7,15,JAMBI
8,32,JAWA BARAT
9,33,JAWA TENGAH


ID Provinsi : 53


Apakah data sudah sesuai ? (Y/N) (default Y) : Y


Apakah data sudah sesuai ? (Y/N) (default Y) : N


,index_right,ID,NAMA_KABUP,NAMA_KECAM,NAMA_DESA
1,4654,4655.0,SUMBA TIMUR,LEWA,KAMBUHAPANG
2,28911,28912.0,SUMBA TIMUR,NGGAHA ORI ANGU,PRAIPAHA
3,55984,55985.0,SUMBA TIMUR,NGGAHA ORI ANGU,PRAIKARANG
4,61090,61091.0,SUMBA TIMUR,NGGAHA ORI ANGU,TANATUKU
5,63191,63192.0,SUMBA TIMUR,KATALA HAMU LINGU,KOMPA PARI
6,79530,79531.0,SUMBA TIMUR,NGGAHA ORI ANGU,KAHIRI
